In [2]:
%matplotlib inline

# USAGE
# python train_vgg.py --dataset animals --model output/smallvggnet.model --label-bin output/smallvggnet_lb.pickle --plot output/smallvggnet_plot.png

# set the matplotlib backend so figures can be saved in the background

#matplotlib.use("Agg")

In [3]:
# import the necessary packages
from pyimagesearch.smallvggnet import SmallVGGNet
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import SGD
from imutils import paths
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import argparse
import random
import pickle
import cv2
import os

Using TensorFlow backend.


In [4]:
# construct the argument parser and parse the arguments
#ap = argparse.ArgumentParser()
#ap.add_argument("--dataset", type=str, default="animals",help="path to input dataset of images")
#ap.add_argument("-m", "--model", type=str, default="output/vgg_nn.model", help="path to output trained model")
#ap.add_argument("-l", "--label-bin", type=str, default="output/vgg_nn_lb.pickle", help="path to output label binarizer")
#ap.add_argument("-p", "--plot", type=str, default="output/vgg_nn_plot.png", help="path to output accuracy/loss plot")
#args = ap.parse_args()

In [5]:
# construct the argument parser and parse the arguments
args = {
  "dataset": "animals", # path to input dataset of images
  "model": "output/vgg_nn.model",    # path to output trained model
  "label-bin": "output/vgg_nn_lb.pickle",# path to output label binarizer
  "plot":"output/vgg_nn_plot.png"       # path to output accuracy/loss plot
}

In [6]:
# initialize the data and labels
print("[INFO] loading images...")
data = []
labels = []

# grab the image paths and randomly shuffle them
imagePaths = sorted(list(paths.list_images(args["dataset"])))
random.seed(42)
random.shuffle(imagePaths)

# loop over the input images
for imagePath in imagePaths:
    # load the image, resize it to 64x64 pixels (the required input spatial dimensions of SmallVGGNet), and store the image in the
    # data list
    image = cv2.imread(imagePath)
    image = cv2.resize(image, (64, 64))
    data.append(image)

    # extract the class label from the image path and update the
    # labels list
    label = imagePath.split(os.path.sep)[-2]
    labels.append(label)

[INFO] loading images...


In [7]:
# scale the raw pixel intensities to the range [0, 1]
data = np.array(data, dtype="float") / 255.0
labels = np.array(labels)

In [8]:
# partition the data into training and testing splits using 75% of
# the data for training and the remaining 25% for testing
(trainX, testX, trainY, testY) = train_test_split(data,labels, test_size=0.25, random_state=42)


 convert the labels from integers to vectors (for 2-class, binary
 classification you should use Keras' to_categorical function
 instead as the scikit-learn's LabelBinarizer will not return a
 vector)

In [9]:
lb = LabelBinarizer()
trainY = lb.fit_transform(trainY)
testY = lb.transform(testY)

In [10]:
# construct the image generator for data augmentation
aug = ImageDataGenerator(rotation_range=30, width_shift_range=0.1, 
                         height_shift_range=0.1, shear_range=0.2, zoom_range=0.2,
                         horizontal_flip=True, fill_mode="nearest")

In [11]:
# initialize our VGG-like Convolutional Neural Network
model = SmallVGGNet.build(width=64, height=64, depth=3, classes=len(lb.classes_))


Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [12]:
# initialize our initial learning rate, # of epochs to train for,
# and batch size
INIT_LR = 0.01
EPOCHS = 75
BS = 32

In [13]:
# initialize the model and optimizer (you'll want to use binary_crossentropy for 2-class classification)
print("[INFO] training network...")
opt = SGD(lr=INIT_LR, decay=INIT_LR / EPOCHS)
model.compile(loss="categorical_crossentropy", optimizer=opt, metrics=["accuracy"])

[INFO] training network...


In [14]:
%%time
# train the network
H = model.fit_generator(aug.flow(trainX, trainY, batch_size=BS),
                        validation_data=(testX, testY), 
                        steps_per_epoch=len(trainX) // BS,
                        epochs=EPOCHS)

Instructions for updating:
Use tf.cast instead.
Epoch 1/75
70/70 [==============================] - 66s 938ms/step - loss: 1.3574 - acc: 0.5259 - val_loss: 1.2133 - val_acc: 0.5773
Epoch 2/75
70/70 [==============================] - 62s 886ms/step - loss: 1.0585 - acc: 0.5810 - val_loss: 0.8417 - val_acc: 0.6453
Epoch 3/75
70/70 [==============================] - 61s 873ms/step - loss: 0.9070 - acc: 0.6175 - val_loss: 1.2626 - val_acc: 0.4600
Epoch 4/75
70/70 [==============================] - 62s 883ms/step - loss: 0.8315 - acc: 0.6267 - val_loss: 0.7422 - val_acc: 0.6640
Epoch 5/75
70/70 [==============================] - 61s 870ms/step - loss: 0.7849 - acc: 0.6339 - val_loss: 0.7370 - val_acc: 0.6573
Epoch 6/75
70/70 [==============================] - 60s 859ms/step - loss: 0.7495 - acc: 0.6465 - val_loss: 0.8271 - val_acc: 0.6213
Epoch 7/75
70/70 [==============================] - 60s 855ms/step - loss: 0.7348 - acc: 0.6545 - val_loss: 0.6462 - val_acc: 0.6827
Epoch 8/75
70/70 [===

70/70 [==============================] - 60s 861ms/step - loss: 0.4626 - acc: 0.7936 - val_loss: 1.1067 - val_acc: 0.6240
Epoch 61/75
70/70 [==============================] - 60s 856ms/step - loss: 0.4694 - acc: 0.7852 - val_loss: 0.5495 - val_acc: 0.7760
Epoch 62/75
70/70 [==============================] - 60s 857ms/step - loss: 0.4455 - acc: 0.7944 - val_loss: 0.5434 - val_acc: 0.7667
Epoch 63/75
70/70 [==============================] - 60s 860ms/step - loss: 0.4433 - acc: 0.7970 - val_loss: 0.5508 - val_acc: 0.7680
Epoch 64/75
70/70 [==============================] - 60s 855ms/step - loss: 0.4467 - acc: 0.7986 - val_loss: 0.5010 - val_acc: 0.8000
Epoch 65/75
70/70 [==============================] - 60s 855ms/step - loss: 0.4912 - acc: 0.7808 - val_loss: 1.8212 - val_acc: 0.5053
Epoch 66/75
70/70 [==============================] - 60s 860ms/step - loss: 0.4519 - acc: 0.7907 - val_loss: 0.5430 - val_acc: 0.7613
Epoch 67/75
70/70 [==============================] - 60s 855ms/step - loss

So it took more than 1hr to train (Wall time: 1h 6min 7s) with validation accuracyof 98%.

In [16]:
# evaluate the network
print("[INFO] evaluating network...")
predictions = model.predict(testX, batch_size=32)
print(classification_report(testY.argmax(axis=1), predictions.argmax(axis=1), target_names=lb.classes_))

[INFO] evaluating network...
              precision    recall  f1-score   support

        cats       0.67      0.76      0.71       236
        dogs       0.71      0.47      0.56       236
       panda       0.81      0.96      0.88       278

   micro avg       0.74      0.74      0.74       750
   macro avg       0.73      0.73      0.72       750
weighted avg       0.73      0.74      0.73       750



We see an overall accuracy of 74% which is better than simple NN. It looks like that panda class has the best accuracy while dogss class has the second best. let's try to see where we get the best accuracy.

In [18]:
# plot the training loss and accuracy
N = np.arange(0, EPOCHS)
plt.style.use("ggplot")
plt.figure()
plt.plot(N, H.history["loss"], label="train_loss")
plt.plot(N, H.history["val_loss"], label="val_loss")
plt.plot(N, H.history["acc"], label="train_acc")
plt.plot(N, H.history["val_acc"], label="val_acc")
plt.title("Training Loss and Accuracy (SmallVGGNet)")
plt.xlabel("Epoch #")
plt.ylabel("Loss/Accuracy")
plt.legend()


NameError: name 'H' is not defined

<Figure size 432x288 with 0 Axes>

We see validation accuracy drops around epoc 65. but, don't see any overfitting.

In [19]:
plt.savefig(args["plot"])

<Figure size 432x288 with 0 Axes>

In [20]:
# save the model and label binarizer to disk
print("[INFO] serializing network and label binarizer...")
model.save(args["model"])
f = open(args["label-bin"], "wb")
f.write(pickle.dumps(lb))
f.close()

[INFO] serializing network and label binarizer...
